## プロジェクトの概要

心疾患（Heart Disease）は、全世界的に主要な死因のひとつであり、早期発見と予測が人命救助において極めて重要である。本プロジェクトは、心疾患の有無を予測するためにロジスティック回帰モデルを構築・活用することを目的としている。

このデータセットは、個人の健康情報（年齢、性別、血圧、コレステロール、血糖値、運動時の心電図反応など）を含み、これらの変数が心疾患の発症にどのように関係しているかを分析するためのものである。

流れは以下の通り
* データの前処理
* 目的変数と説明変数の分離
* ACIによる特徴量選択[上位10データを選択]
* train_data test_dataに分割(8:2)
* 予測と評価

## 工夫点
* ACIによる特徴量エンジニアリングにより不要な説明変数を削除した点
デフォルトのモデルではaccuracyは高いがrecll,f1値が低いため以下を導入した
* 正解データの偏りをなくすためweight _decay=ballanced
* L2正則化の導入


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import scipy.stats as stats

#前処理
df_framingham = pd.read_csv("../data/framingham.csv")  
df_clean = df_framingham.dropna()
X_full = df_clean.drop(columns=["TenYearCHD"])
y = df_clean["TenYearCHD"]

#ACI
correlations = {}
for col in X_full.columns:
    try:
        corr, _ = stats.pearsonr(X_full[col], y)
        correlations[col] = abs(corr)  
    except:
        correlations[col] = 0

sorted_features = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
top_features = [feature for feature, _ in sorted_features[:10]]

print("選ばれた特徴量（ACI上位5）：", top_features)

#学習データの再構築
X_selected = X_full[top_features]

#学習データの分割
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# モデルの学習
model = LogisticRegression(
    penalty='l2',       # L2正則化
    C=1.0,              
    class_weight='balanced',
    max_iter=1000
)
model.fit(X_train, y_train)

#予測と評価
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nAccuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)


選ばれた特徴量（ACI上位5）： ['age', 'sysBP', 'prevalentHyp', 'diaBP', 'glucose', 'diabetes', 'male', 'totChol', 'BPMeds', 'BMI']

Accuracy: 0.6803

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.69      0.78       610
           1       0.29      0.65      0.40       122

    accuracy                           0.68       732
   macro avg       0.60      0.67      0.59       732
weighted avg       0.80      0.68      0.72       732



/home/gen/anaconda3/envs/keiei/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
df_clean.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [16]:
X_full.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0


In [17]:
y.head()

0    0
1    0
2    0
3    1
4    0
Name: TenYearCHD, dtype: int64